In [15]:
from influxdb import InfluxDBClient
import pandas as pd
import numpy as np
from skimpy import skim
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import PolynomialFeatures
import ewtpy

In [16]:
import sys
sys.path.append('..')
import fx

In [17]:
data = fx.pull_data()

In [18]:
X_train, y_train, X_test, y_test = fx.data_splitting(data)

In [24]:
testing = Pipeline(steps=[
    ("col_transformer", ColumnTransformer(transformers=[
        ("Speed", fx.EmpiricalWaveletTransform(level=5, log=True), ["Speed"]),
        ("Direction", fx.CompassToCartesianTransformer(), ["Direction"]),
        ], remainder="drop")),
    ("LinRegModel", LinearRegression())
])

In [25]:
testing.fit(X_train, y_train)

Pipeline(steps=[('col_transformer',
                 ColumnTransformer(transformers=[('Speed',
                                                  EmpiricalWaveletTransform(),
                                                  ['Speed']),
                                                 ('Direction',
                                                  CompassToCartesianTransformer(),
                                                  ['Direction'])])),
                ('LinRegModel', LinearRegression())])

In [26]:
testing.predict(X_test)

array([25.46460658, 21.90625695, 13.71571022,  8.55755131,  7.47229838,
       14.81275044, 18.02949639, 21.06357684, 24.16540936, 26.31226069,
       29.06896185, 29.92750218, 31.03326301, 28.64053014, 22.29544161,
       16.18707065, 13.99051868, 14.80834141, 16.49041949, 17.90960767,
       18.7343383 , 18.74877348, 19.61432553, 22.03195302, 24.92040107,
       23.7758615 , 23.51030889, 22.72734956, 20.98260952, 20.64163374,
       19.73525417, 21.70874577, 21.13244727, 20.4430077 , 21.07657922,
       21.14572801, 22.97877001, 24.70749085, 25.18134641, 24.09727686,
       23.08996732, 21.69493297, 21.67823748, 21.41111842, 21.09316328,
       14.3800937 , 17.66186033, 15.7707028 , 16.24069681, 18.65132482,
       20.66870747, 21.3938447 , 22.13585976, 23.74091576, 26.85037102,
       28.67302976, 30.59090411, 31.30608696, 27.57318138, 25.74295205,
       25.42751453, 25.75371652, 28.14399596, 29.17421708, 27.17315627,
       20.86015642, 14.5837715 , 11.13692256, 13.90127421, 15.99

In [27]:
fx.RMSE(testing.predict(X_test), y_test)

6.367403118603512

In [28]:
fx.MSE(testing.predict(X_test), y_test)

40.54382247480173

In [33]:
plot_df = fx.create_timestamps(testing.predict(X_test), X_test, y_test)
plot_df

,predict,actual
2023-02-01 12:00:00+00:00,25.464607,16.627726
2023-02-01 15:00:00+00:00,21.906257,17.442904
2023-02-01 18:00:00+00:00,13.715710,9.886359
2023-02-01 21:00:00+00:00,8.557551,5.637801
2023-02-02 00:00:00+00:00,7.472298,3.519408
...,...,...
2023-02-15 18:00:00+00:00,21.844684,23.819081
2023-02-15 21:00:00+00:00,20.676483,21.940404
2023-02-16 00:00:00+00:00,19.629546,19.402905
2023-02-16 03:00:00+00:00,18.925123,16.152971


In [35]:
# plot plot_df
fig = px.line(plot_df, title="Predicted vs. Actual Generation")
fig.show()